# Polymarket Market Making Backtest

Market-making strategy backtest for Polymarket prediction markets.

**Strategy**: Two-sided quoting (bid + ask) around mid-price to capture spread.
Adapted from [poly-maker](https://github.com/warproxxx/poly-maker) with Polymarket-specific fee model.

**Data sources**:
1. Synthetic data (for strategy validation)
2. CLOB API `/prices-history` (hourly candles)
3. `archive.pmxt.dev` orderbook snapshots (Parquet)

**Fee model**: Maker orders (limit orders) = 0% fee + rebate income

## 1. Setup

In [ ]:
# Google Colab setup
import os
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in str(get_ipython())

if IN_COLAB:
    # Mount Google Drive for persistent storage
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_DIR = '/content/drive/MyDrive/polymarket-data'
    os.makedirs(DATA_DIR, exist_ok=True)

    # Clone repo
    !git clone https://github.com/jsseoi/polymarket-trading-bot.git /content/polymarket-trading-bot 2>/dev/null || \
        (cd /content/polymarket-trading-bot && git pull)
    os.chdir('/content/polymarket-trading-bot')

    # Install dependencies
    !pip install -q pandas numpy matplotlib pyarrow optuna requests pyyaml structlog ratelimit
else:
    # Local development
    DATA_DIR = 'data'
    os.makedirs(DATA_DIR, exist_ok=True)

import sys
sys.path.insert(0, '.')

print(f'Data directory: {DATA_DIR}')
print(f'Working directory: {os.getcwd()}')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from src.strategies.market_making import (
    MarketMakingStrategy,
    MarketMakingParams,
    FEE_POLITICAL,
    FEE_CRYPTO,
    FEE_SPORTS,
)
from src.backtesting.mm_engine import (
    MarketMakingEngine,
    MMBacktestConfig,
    MMBacktestResult,
)
from src.backtesting.engine import BacktestEngine, BacktestConfig

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print('Imports OK')

## 2. Synthetic Data Backtest (Strategy Validation)

In [ ]:
# Generate synthetic data
engine = MarketMakingEngine()
count = engine.generate_mm_synthetic_data(
    num_markets=30,
    days=90,
    snapshots_per_day=4,
    seed=42,
)
print(f'Generated {count} snapshots across {len(engine.market_data)} markets')

In [ ]:
# Default parameters (conservative)
params = MarketMakingParams(
    min_spread=0.02,        # 2 cent minimum spread
    trade_size=50.0,        # $50 per order
    max_size=200.0,         # $200 max per market
    stop_loss_pct=-5.0,     # -5% stop-loss
    take_profit_pct=2.0,    # +2% take-profit
    volatility_threshold=0.10,
    sleep_period_hours=1.0,
    min_liquidity=5000.0,
    min_volume_24h=10000.0,
    fee_config=FEE_POLITICAL,
)

strategy = MarketMakingStrategy(params)

config = MMBacktestConfig(
    start_date=datetime.now() - timedelta(days=90),
    end_date=datetime.now(),
    initial_capital=1000.0,  # ~10만원
    fill_aggression=0.5,
    use_random_fills=True,
)

result = engine.run_mm(strategy, config)
print(result.mm_summary())

In [ ]:
# Equity curve visualization
eq_df = pd.DataFrame(result.equity_curve)
eq_df['date'] = pd.to_datetime(eq_df['date'])

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Equity curve
axes[0].plot(eq_df['date'], eq_df['equity'], color='#2196F3', linewidth=1.5)
axes[0].axhline(y=config.initial_capital, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
axes[0].set_ylabel('Equity ($)')
axes[0].set_title('Market Making Strategy - Equity Curve')
axes[0].legend()

# Drawdown
axes[1].fill_between(eq_df['date'], 0, -eq_df['drawdown'], color='#F44336', alpha=0.3)
axes[1].plot(eq_df['date'], -eq_df['drawdown'], color='#F44336', linewidth=1)
axes[1].set_ylabel('Drawdown ($)')
axes[1].set_title('Drawdown')

# Active positions
axes[2].bar(eq_df['date'], eq_df['positions'], color='#4CAF50', alpha=0.6)
axes[2].set_ylabel('Open Positions')
axes[2].set_title('Active Market Positions')
axes[2].set_xlabel('Date')

plt.tight_layout()
plt.show()

In [ ]:
# Fill analysis
fills_df = pd.DataFrame(result.fills)
if len(fills_df) > 0:
    fills_df['timestamp'] = pd.to_datetime(fills_df['timestamp'])

    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    # Fill distribution by side
    side_counts = fills_df['side'].value_counts()
    axes[0].bar(side_counts.index, side_counts.values, color=['#4CAF50', '#F44336'])
    axes[0].set_title('Fills by Side')
    axes[0].set_ylabel('Count')

    # Price distribution of fills
    axes[1].hist(fills_df['price'], bins=30, color='#2196F3', alpha=0.7, edgecolor='white')
    axes[1].set_title('Fill Price Distribution')
    axes[1].set_xlabel('Price')

    # Fills over time
    daily_fills = fills_df.groupby(fills_df['timestamp'].dt.date).size()
    axes[2].bar(range(len(daily_fills)), daily_fills.values, color='#9C27B0', alpha=0.7)
    axes[2].set_title('Fills per Day')
    axes[2].set_xlabel('Day')

    plt.tight_layout()
    plt.show()

    print(f'Total fills: {len(fills_df)}')
    print(f'Buy fills: {len(fills_df[fills_df.side == "BUY"])}')
    print(f'Sell fills: {len(fills_df[fills_df.side == "SELL"])}')
    print(f'Unique markets: {fills_df.market_id.nunique()}')
else:
    print('No fills generated')

## 3. Parameter Sensitivity Analysis

In [ ]:
# Test different spread thresholds
spreads = [0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05]
results_by_spread = []

for spread in spreads:
    p = MarketMakingParams(
        min_spread=spread,
        trade_size=50.0,
        max_size=200.0,
        stop_loss_pct=-5.0,
        take_profit_pct=2.0,
        fee_config=FEE_POLITICAL,
    )
    s = MarketMakingStrategy(p)
    r = engine.run_mm(s, config)
    results_by_spread.append({
        'spread': spread,
        'return_pct': r.total_return_pct,
        'sharpe': r.sharpe_ratio,
        'win_rate': r.win_rate,
        'trades': r.total_trades,
        'max_dd': r.max_drawdown_pct,
        'fill_rate': r.fill_rate,
    })
    print(f'  spread={spread:.3f}: return={r.total_return_pct:.1%}, sharpe={r.sharpe_ratio:.2f}, fills={r.total_trades}')

spread_df = pd.DataFrame(results_by_spread)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].plot(spread_df['spread'], spread_df['return_pct'], 'o-', color='#2196F3')
axes[0].set_xlabel('Min Spread')
axes[0].set_ylabel('Return %')
axes[0].set_title('Return vs Spread Threshold')

axes[1].plot(spread_df['spread'], spread_df['sharpe'], 's-', color='#4CAF50')
axes[1].set_xlabel('Min Spread')
axes[1].set_ylabel('Sharpe Ratio')
axes[1].set_title('Sharpe vs Spread Threshold')

axes[2].plot(spread_df['spread'], spread_df['trades'], 'D-', color='#F44336')
axes[2].set_xlabel('Min Spread')
axes[2].set_ylabel('Total Trades')
axes[2].set_title('Trade Count vs Spread Threshold')

plt.tight_layout()
plt.show()

In [ ]:
# Test different position sizes
trade_sizes = [20, 30, 50, 75, 100, 150]
results_by_size = []

for ts in trade_sizes:
    p = MarketMakingParams(
        min_spread=0.02,
        trade_size=float(ts),
        max_size=float(ts) * 4,
        stop_loss_pct=-5.0,
        take_profit_pct=2.0,
        fee_config=FEE_POLITICAL,
    )
    s = MarketMakingStrategy(p)
    r = engine.run_mm(s, config)
    results_by_size.append({
        'trade_size': ts,
        'return_pct': r.total_return_pct,
        'sharpe': r.sharpe_ratio,
        'max_dd': r.max_drawdown_pct,
        'trades': r.total_trades,
        'volume': r.total_volume,
    })
    print(f'  size=${ts}: return={r.total_return_pct:.1%}, sharpe={r.sharpe_ratio:.2f}, dd={r.max_drawdown_pct:.1%}')

size_df = pd.DataFrame(results_by_size)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(size_df['trade_size'], size_df['return_pct'], 'o-', label='Return %', color='#2196F3')
ax2 = ax.twinx()
ax2.plot(size_df['trade_size'], size_df['max_dd'], 's--', label='Max Drawdown %', color='#F44336')
ax.set_xlabel('Trade Size ($)')
ax.set_ylabel('Return %', color='#2196F3')
ax2.set_ylabel('Max Drawdown %', color='#F44336')
ax.set_title('Return vs Risk by Position Size')
fig.legend(loc='upper left', bbox_to_anchor=(0.12, 0.88))
plt.tight_layout()
plt.show()

## 4. Historical Data Backtest

In [ ]:
# Collect real data (run this once, then use cached data)
from src.data.mm_collector import MMDataCollector

collector = MMDataCollector(cache_dir=f'{DATA_DIR}/cache')

# Find suitable markets
markets = collector.find_mm_markets(
    min_volume_24h=10_000,
    min_liquidity=5_000,
    max_markets=30,
)

if markets:
    print(f'\nTop markets by liquidity:')
    for m in markets[:10]:
        print(f'  ${m.liquidity:>12,.0f} | vol24h ${m.volume_24h:>10,.0f} | {m.question[:55]}')
else:
    print('No suitable markets found. Using synthetic data only.')

In [ ]:
# Collect price history
HIST_FILE = f'{DATA_DIR}/mm_historical.json'

if markets:
    snapshots = collector.collect_price_history(markets[:20], days=60)
    if snapshots:
        collector.save_snapshots(snapshots, HIST_FILE)
        print(f'Saved {len(snapshots)} snapshots')
    else:
        print('No snapshots collected from API')
else:
    print('Skipping - no markets available')

In [ ]:
# Run backtest on historical data
import os

if os.path.exists(HIST_FILE):
    hist_engine = MarketMakingEngine()
    count = hist_engine.load_data(HIST_FILE)
    print(f'Loaded {count} historical snapshots')

    # Find actual date range in data
    dates = [s.timestamp for s in hist_engine.all_snapshots]
    data_start = min(dates)
    data_end = max(dates)
    print(f'Date range: {data_start.date()} to {data_end.date()}')

    hist_strategy = MarketMakingStrategy(MarketMakingParams(
        min_spread=0.02,
        trade_size=50.0,
        max_size=200.0,
        stop_loss_pct=-5.0,
        take_profit_pct=2.0,
        fee_config=FEE_POLITICAL,
    ))

    hist_config = MMBacktestConfig(
        start_date=data_start,
        end_date=data_end,
        initial_capital=1000.0,
        fill_aggression=0.4,  # More conservative for real data
        use_random_fills=True,
    )

    hist_result = hist_engine.run_mm(hist_strategy, hist_config)
    print(hist_result.mm_summary())
else:
    print(f'No historical data file at {HIST_FILE}')
    print('Run the data collection cell first, or use synthetic data.')

## 5. Fee Model Comparison

In [ ]:
# Compare performance across different fee regimes
fee_configs = {
    'Political (0% fee)': FEE_POLITICAL,
    'Crypto (taker 1.56% max)': FEE_CRYPTO,
    'Sports (taker 0.44% max)': FEE_SPORTS,
}

fee_results = {}
for name, fc in fee_configs.items():
    p = MarketMakingParams(
        min_spread=0.02,
        trade_size=50.0,
        max_size=200.0,
        fee_config=fc,
    )
    s = MarketMakingStrategy(p)
    r = engine.run_mm(s, config)
    fee_results[name] = r
    print(f'{name}:')
    print(f'  Return: {r.total_return_pct:.1%} | Sharpe: {r.sharpe_ratio:.2f} | '
          f'Rebates: ${r.total_maker_rebates:.2f} | Fills: {r.total_trades}')

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

names = list(fee_results.keys())
returns = [fee_results[n].total_return_pct for n in names]
rebates = [fee_results[n].total_maker_rebates for n in names]

colors = ['#4CAF50', '#2196F3', '#FF9800']
axes[0].bar(range(len(names)), returns, color=colors, alpha=0.8)
axes[0].set_xticks(range(len(names)))
axes[0].set_xticklabels(names, rotation=15)
axes[0].set_ylabel('Return %')
axes[0].set_title('Return by Fee Regime')

axes[1].bar(range(len(names)), rebates, color=colors, alpha=0.8)
axes[1].set_xticks(range(len(names)))
axes[1].set_xticklabels(names, rotation=15)
axes[1].set_ylabel('Maker Rebates ($)')
axes[1].set_title('Maker Rebate Income by Fee Regime')

plt.tight_layout()
plt.show()

## 6. Polymarket Fee Model Visualization

In [ ]:
# Visualize the fee curve
prices = np.linspace(0.01, 0.99, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Taker fees
crypto_fees = [FEE_CRYPTO.taker_fee(p) * 100 for p in prices]
sports_fees = [FEE_SPORTS.taker_fee(p) * 100 for p in prices]

axes[0].plot(prices, crypto_fees, label='Crypto (5/15min)', color='#2196F3', linewidth=2)
axes[0].plot(prices, sports_fees, label='Sports (NCAAB, Serie A)', color='#FF9800', linewidth=2)
axes[0].axhline(y=0, color='#4CAF50', linewidth=2, label='Political (0%)')
axes[0].set_xlabel('Price (probability)')
axes[0].set_ylabel('Taker Fee (%)')
axes[0].set_title('Polymarket Taker Fee by Price Level')
axes[0].legend()
axes[0].set_xlim(0, 1)

# Maker rebates
crypto_rebates = [FEE_CRYPTO.maker_rebate(p) * 100 for p in prices]
sports_rebates = [FEE_SPORTS.maker_rebate(p) * 100 for p in prices]

axes[1].plot(prices, crypto_rebates, label='Crypto rebate (20%)', color='#2196F3', linewidth=2)
axes[1].plot(prices, sports_rebates, label='Sports rebate (25%)', color='#FF9800', linewidth=2)
axes[1].set_xlabel('Price (probability)')
axes[1].set_ylabel('Maker Rebate (%)')
axes[1].set_title('Maker Rebate Income by Price Level')
axes[1].legend()
axes[1].set_xlim(0, 1)

plt.tight_layout()
plt.show()

print('Key takeaways:')
print(f'  Crypto max taker fee: {FEE_CRYPTO.taker_fee(0.5)*100:.2f}% at 50c')
print(f'  Sports max taker fee: {FEE_SPORTS.taker_fee(0.5)*100:.2f}% at 50c')
print(f'  Crypto maker rebate at 50c: {FEE_CRYPTO.maker_rebate(0.5)*100:.4f}%')
print(f'  Sports maker rebate at 50c: {FEE_SPORTS.maker_rebate(0.5)*100:.4f}%')
print(f'  Political markets: 0% fee, 0% rebate')

## 7. Multiple Run Monte Carlo (Robustness Check)

In [ ]:
# Run multiple times with random fills to check robustness
n_runs = 50
mc_results = []

base_params = MarketMakingParams(
    min_spread=0.02,
    trade_size=50.0,
    max_size=200.0,
    stop_loss_pct=-5.0,
    take_profit_pct=2.0,
    fee_config=FEE_POLITICAL,
)

mc_config = MMBacktestConfig(
    start_date=datetime.now() - timedelta(days=90),
    end_date=datetime.now(),
    initial_capital=1000.0,
    fill_aggression=0.5,
    use_random_fills=True,  # Randomness in fill simulation
)

for i in range(n_runs):
    s = MarketMakingStrategy(base_params)
    r = engine.run_mm(s, mc_config)
    mc_results.append({
        'run': i,
        'return_pct': r.total_return_pct,
        'sharpe': r.sharpe_ratio,
        'max_dd': r.max_drawdown_pct,
        'trades': r.total_trades,
        'win_rate': r.win_rate,
        'fill_rate': r.fill_rate,
    })

mc_df = pd.DataFrame(mc_results)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].hist(mc_df['return_pct'], bins=20, color='#2196F3', alpha=0.7, edgecolor='white')
axes[0].axvline(mc_df['return_pct'].median(), color='red', linestyle='--', label=f'Median: {mc_df["return_pct"].median():.1%}')
axes[0].set_xlabel('Return %')
axes[0].set_title(f'Return Distribution ({n_runs} runs)')
axes[0].legend()

axes[1].hist(mc_df['sharpe'], bins=20, color='#4CAF50', alpha=0.7, edgecolor='white')
axes[1].axvline(mc_df['sharpe'].median(), color='red', linestyle='--', label=f'Median: {mc_df["sharpe"].median():.2f}')
axes[1].set_xlabel('Sharpe Ratio')
axes[1].set_title('Sharpe Ratio Distribution')
axes[1].legend()

axes[2].hist(mc_df['max_dd'], bins=20, color='#F44336', alpha=0.7, edgecolor='white')
axes[2].axvline(mc_df['max_dd'].median(), color='red', linestyle='--', label=f'Median: {mc_df["max_dd"].median():.1%}')
axes[2].set_xlabel('Max Drawdown %')
axes[2].set_title('Max Drawdown Distribution')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f'Monte Carlo Summary ({n_runs} runs):')
print(f'  Return: {mc_df["return_pct"].mean():.1%} +/- {mc_df["return_pct"].std():.1%}')
print(f'  Sharpe: {mc_df["sharpe"].mean():.2f} +/- {mc_df["sharpe"].std():.2f}')
print(f'  Max DD: {mc_df["max_dd"].mean():.1%} +/- {mc_df["max_dd"].std():.1%}')
print(f'  Win Rate: {mc_df["win_rate"].mean():.1%} +/- {mc_df["win_rate"].std():.1%}')
print(f'  P(positive return): {(mc_df["return_pct"] > 0).mean():.1%}')

---

## Next Steps

1. **Collect real data**: Run the data collection cells to get actual Polymarket price history
2. **Run Optuna optimization**: See `notebooks/mm_optimize.ipynb` for parameter optimization
3. **Paper trading**: Connect to live market data for virtual trading validation
4. **Phase 2**: Live deployment with real capital after 1-2 weeks of paper trading